In [31]:
stub='aus'

In [32]:
#laps_fn='/Users/ajh59/Dropbox/F1data/2017/eventTiming/AUS/P1 - Lap Times'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_q0_timing_qualifyingsessionlaptimes_v01'#'lap_analysis_5'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_p3_timing_thirdpracticesessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_p2_timing_secondpracticesessionlaptimes_v01_1'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_p1_timing_firstpracticesessionlaptimes_v01'


laps_out_fn='{}18_qualilaptimes.csv'.format(stub)
laps_out_fn='{}18_p1laptimes.csv'.format(stub)
#laps_out_fn='{}17_P1laptimes.csv'.format(stub)

In [33]:
import lxml.etree
import pandas as pd
import os

def xmlify(fn):
    cmd = 'pdftohtml -xml -nodrm -zoom 1.5 -enc UTF-8 -noframes %s "%s" "%s"' % (
        '',fn+'.pdf', os.path.splitext(fn+'.xml')[0])
    # can't turn off output, so throw away even stderr yeuch
    cmd + " >/dev/null 2>&1"
    os.system(cmd)

def loader(fn):
    fnx=fn+'.xml'
    xmldata = open(fn+'.xml','r').read()
    root = lxml.etree.fromstring(xmldata)
    pages = list(root)
    return pages, root

def loader2018(fn):
    fnx=fn+'.xml'
    #xmldata = open(fn+'.xml','r').read()
    xmldata = open(fn+'.xml','rb').read()
    root = lxml.etree.fromstring(xmldata)
    pages = list(root)
    return pages, root

In [34]:
loader = loader2018

In [35]:
xmlify(laps_fn)

In [36]:
def patcher(laps_fn):
    !sed -i.'' -E 's/(<text .*>)([0-9]+) <b>(.*)<\/b><\/text>/\1\2<\/text>\1<b>\3<\/b><\/text>/g' "{laps_fn}.xml"


In [37]:
patcher(laps_fn)

In [38]:
pages,root=loader(laps_fn)

In [39]:
def flatten(el):           
    result = [ (el.text or "") ]
    for sel in el:
        result.append(flatten(sel))
        result.append(sel.tail or "")
    return "".join(result)

def laptimes_table(pages):
    rows=[]

    data=[]

    for page in pages:
        scraping=False
        prevrow=0
        newpair=False
        for el in page:
            txt=flatten(el).strip()
            tmp={'text':txt}
            if 'left' in el.attrib and el.attrib['left'] is not None:
                for attr in el.attrib:
                    tmp[attr]=el.attrib[attr]
                #print txt, el.attrib
                rows.append(tmp)

                prevrow=int(el.attrib['top'])
                if scraping:
                    if not newpair:
                        newpair=txt
                    else:
                        data.append( (newpair,txt) )
                        newpair=False

                if not scraping and (txt.startswith('2017 FORMULA 1') or txt.startswith('FORMULA 1')):
                    scraping=True

    return data,pd.DataFrame.from_records(rows)

In [40]:
def varasint(cand, retval=''):
    ''' Function to determine if we can cast something as an int.
        If we can, do so and return it, otherwise return retval, or, if retval=='asis', the original value. '''
    retval = int(cand) if (int(cand) if isinstance(cand, str) and cand.isdigit() else False) or isinstance(cand, int) else retval
    if retval=='asis':
        return cand
    elif retval=='boolean':
        return isinstance(retval, int)
    return retval

def laptimes_table2018(pages):
    rows=[]

    data=[]

    for page in pages:
        scraping=True
        prevrow=0
        newpair=False
        for el in page:
            txt=flatten(el).strip()
            tmp={'text':txt}
            if 'left' in el.attrib and el.attrib['left'] is not None:
                for attr in el.attrib:
                    tmp[attr]=el.attrib[attr]
                #print txt, el.attrib
                rows.append(tmp)

                prevrow=int(el.attrib['top'])
                if scraping:
                    if not newpair:
                        newpair=txt
                    else:
                        data.append( (newpair,txt) )
                        newpair=False

                if not scraping and (txt.startswith('2018 FORMULA 1') or txt.startswith('FORMULA 1')):
                    scraping=True

        #last two rows:
        #- name of GP eg FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX - Melbourne
        #- Sheet (eg Qualifying Session Lap Times)
    data = [d for d in data if d[0].startswith('NO') or varasint( d[0].split(' ')[0], 'boolean')]
    return data,pd.DataFrame.from_records(rows)

In [41]:
laptimes_table = laptimes_table2018

In [42]:
data,df=laptimes_table2018(pages)

In [43]:
data,df=laptimes_table(pages)
df[:15]

,font,height,left,src,text,top,width
0,NaN,128,36,/Users/ajh59/Downloads/2018_01_aus_f1_p1_timin...,,33,285
1,NaN,86,693,/Users/ajh59/Downloads/2018_01_aus_f1_p1_timin...,,61,150
2,0,17,35,NaN,2,269,7
3,1,17,71,NaN,Stoffel VANDOORNE,269,106
4,1,17,50,NaN,NO,298,16
5,1,17,113,NaN,TIME,298,27
6,1,17,56,NaN,1 P,316,20
7,0,17,101,NaN,12:00:57,316,46
8,1,17,56,NaN,2 P,334,20
9,0,17,94,NaN,36:09.001,334,52


In [44]:
#data

In [45]:
def _grab_laptimes(data):
    name=''
    lap=''
    times=[]
    inlaps=False
    tmp={}
    earliest=None
    for pair in data:
        print(pair)
        if pair==('NO','NO') or pair==('LAP','LAP'):
            if 'laps' in tmp: tmp['laps'].pop()
            number,name=prevpair
            if tmp!={}: times.append(tmp)
            tmp={'name':name,'number':number, 'laps':[]}
        elif pair==('TIME','TIME'):
            inlaps=True
        elif inlaps:
            if earliest is None:
                earliest=(tmp['name'],pair[1])
            elif len(tmp['laps'])==0:
                if earliest[1]>pair[1]:
                    earliest=(tmp['name'],pair[1])
            tmp['laps'].append(pair)

        prevpair=pair
    #Catch tailender
    if tmp!={}: times.append(tmp)
    return earliest,times

def _grab_laptimes2018(data):
    name=''
    lap=''
    times=[]
    inlaps=False
    tmp={}
    earliest=None
    named = False
    for pair in data:
        print(pair)
        if pair==('NO','TIME') and not named:
            if 'laps' in tmp and tmp['laps']: tmp['laps'].pop()
            number,name=prevpair
            if tmp!={}: times.append(tmp)
            tmp={'name':name,'number':number, 'laps':[]}
            named = True
            inlaps=True
        elif pair==('NO','TIME'):
            inlaps=True
            named = False
        elif inlaps:
            if earliest is None:
                earliest=(tmp['name'],pair[1])
            elif len(tmp['laps'])==0:
                if earliest[1]>pair[1]:
                    earliest=(tmp['name'],pair[1])
            tmp['laps'].append(pair)

        prevpair=pair
    #Catch tailender
    if tmp!={}: times.append(tmp)
    return earliest,times

In [46]:
_grab_laptimes = _grab_laptimes2018

In [47]:
earliest,laptimes=_grab_laptimes(data)
earliest

('2', 'Stoffel VANDOORNE')
('NO', 'TIME')
('1 P', '12:00:57')
('2 P', '36:09.001')
('3 P', '19:55.375')
('4', '1:44.092')
('5', '1:29.558')
('6 P', '17:14.718')
('7', '2:00.978')
('8', '1:27.797')
('NO', 'TIME')
('9', '1:35.900')
('10', '1:26.541')
('11', '1:36.092')
('12', '1:38.273')
('13', '1:26.482')
('14', '1:28.045')
('15', '2:28.469')
('3', 'Daniel RICCIARDO')
('NO', 'TIME')
('1 P', '12:01:30')
('2 P', '18:34.696')
('3', '1:43.443')
('4', '1:27.608')
('5', '1:48.849')
('6', '1:26.299')
('7', '1:49.175')
('8', '1:25.462')
('9', '1:51.096')
('10', '1:25.063')
('11 P', '24:30.413')
('12', '1:45.258')
('13', '1:25.130')
('NO', 'TIME')
('14', '1:52.826')
('15 P', '1:52.838')
('16', '1:51.489')
('17', '1:25.565')
('18', '1:55.108')
('19 P', '12:30.018')
('20', '1:58.271')
('21', '1:29.347')
('22', '1:29.496')
('23', '1:29.061')
('24', '1:29.042')
('25', '2:05.780')
('5', 'Sebastian VETTEL')
('NO', 'TIME')
('1 P', '12:00:09')
('2 P', '20:18.710')
('3', '1:44.808')
('4', '1:30.108')
('5

('Sebastian VETTEL', '12:00:09')

In [48]:
names = [ '"{}"=\'\''.format(l['name']) for l in laptimes]
print(', '.join(names))

"Stoffel VANDOORNE"='', "Daniel RICCIARDO"='', "Sebastian VETTEL"='', "Kimi RAIKKONEN"='', "Romain GROSJEAN"='', "Marcus ERICSSON"='', "Pierre GASLY"='', "Sergio PEREZ"='', "Fernando ALONSO"='', "Charles LECLERC"='', "Lance STROLL"='', "Kevin MAGNUSSEN"='', "Nico HULKENBERG"='', "Brendon HARTLEY"='', "Esteban OCON"='', "Max VERSTAPPEN"='', "Sergey SIROTKIN"='', "Lewis HAMILTON"='', "Carlos SAINZ"='', "Valtteri BOTTAS"=''


In [49]:
import datetime
#http://tgs.github.io/nptime/
from nptime import nptime

def nptimify(t):
    tt=t.split(':')
    if len(tt)==3:
        h=int(tt[0])
        m=int(tt[1])
        if len(tt[2].split('.'))==2:
            s=int(tt[2].split('.')[0])
            ms=int(tt[2].split('.')[1])
        else:
            s=int(t.split(':')[2])
            ms=0
    elif len(tt)==2:
        h=int(tt[0])
        m=int(tt[1])
        if len(tt[2].split('.'))==2:
            s=int(tt[2].split('.')[0])
            ms=int(tt[2].split('.')[1])
        else:
            s=int(t.split(':')[2])
            ms=0
    return nptime(h, m, s, ms)

def npreltimify(t,earliest="0:0:0"):
    return nptimify(t) - nptimify(earliest)

def nprebase(delta,earliest="0:0:0"):
    return delta + nptimify(earliest)

#def nprebaseElapsed()

#Preferred time format
def formatTime(t):
    return float("%.3f" % t)
# Accept times in the form of hh:mm:ss.ss or mm:ss.ss
# Return the equivalent number of seconds
def getTime(ts):
    print(ts)
    t=ts.strip()
    t=ts.split(':')
    if len(t)==3:
        tm=60*int(t[0])+60*int(t[1])+float(t[2])
    elif len(t)==2:
        tm=60*int(t[0])+float(t[1])
    else:
        tm=float(t[0])
    return formatTime(tm)

In [50]:
def _get_rebased_laptimes(times,earliest): 
    rr=[]
    for driver in times:
        for lap in driver['laps']:
            if lap[0].split(' ')[0]=='1':
                b=nprebase(npreltimify(lap[1],earliest[1])).isoformat()
            else:
                b=lap[1]
            rr.append({'name':driver['name'],
                       'number':driver['number'],
                       'lapNumber':lap[0].split(' ')[0],
                       'pit': True if lap[0].endswith('P') else False,
                       'laptime':b,
                       'stime':getTime(b)})
    return pd.DataFrame(rr)

In [51]:
#Patch for italy 2017 where there was a restart
laptimespatch=[]
for lapset in laptimes:
    tmp={'name':lapset['name'],'number':lapset['number'],'laps':[]}
    cnt=0
    started =False
    for (lapnum,laptime) in lapset['laps']:
        if laptime.startswith('16'):
            started =True 
        if started:
            cnt=cnt+1
            tmp['laps'].append( (str(cnt),laptime) )
    laptimespatch.append(tmp)

newearliest=None
for lapset in laptimes:
    if newearliest is None:
        newearliest=(lapset['name'],lapset['laps'][0][1])
    else:
        if len(lapset['laps']) and lapset['laps'][0][1]<newearliest[1]:
            newearliest=(lapset['name'],lapset['laps'][0][1])
newearliest
#earliest=newearliest

#laptimes=laptimespatch

('Sebastian VETTEL', '12:00:09')

In [52]:
lapset['laps'][0][1]

'12:04:34'

In [53]:
for l in laptimes:
    if l['laps']: print(l['laps'][0])

('1 P', '12:00:57')
('1 P', '12:01:30')
('1 P', '12:00:09')
('1 P', '12:01:13')
('1 P', '12:00:13')
('1 P', '12:00:20')
('1 P', '12:01:43')
('1 P', '12:02:35')
('1 P', '12:00:30')
('1 P', '12:01:19')
('1 P', '12:03:01')
('1 P', '12:01:36')
('1 P', '12:01:17')
('1 P', '12:01:33')
('1 P', '12:01:57')
('1 P', '12:02:32')
('1 P', '12:02:22')
('1 P', '12:05:59')
('1 P', '12:02:06')
('1 P', '12:04:34')


In [54]:
laptimes

[{'laps': [('1 P', '12:00:57'),
   ('2 P', '36:09.001'),
   ('3 P', '19:55.375'),
   ('4', '1:44.092'),
   ('5', '1:29.558'),
   ('6 P', '17:14.718'),
   ('7', '2:00.978'),
   ('8', '1:27.797'),
   ('9', '1:35.900'),
   ('10', '1:26.541'),
   ('11', '1:36.092'),
   ('12', '1:38.273'),
   ('13', '1:26.482'),
   ('14', '1:28.045'),
   ('15', '2:28.469')],
  'name': 'Stoffel VANDOORNE',
  'number': '2'},
 {'laps': [('1 P', '12:01:30'),
   ('2 P', '18:34.696'),
   ('3', '1:43.443'),
   ('4', '1:27.608'),
   ('5', '1:48.849'),
   ('6', '1:26.299'),
   ('7', '1:49.175'),
   ('8', '1:25.462'),
   ('9', '1:51.096'),
   ('10', '1:25.063'),
   ('11 P', '24:30.413'),
   ('12', '1:45.258'),
   ('13', '1:25.130'),
   ('14', '1:52.826'),
   ('15 P', '1:52.838'),
   ('16', '1:51.489'),
   ('17', '1:25.565'),
   ('18', '1:55.108'),
   ('19 P', '12:30.018'),
   ('20', '1:58.271'),
   ('21', '1:29.347'),
   ('22', '1:29.496'),
   ('23', '1:29.061'),
   ('24', '1:29.042'),
   ('25', '2:05.780')],
  'name

In [55]:
ddx=_get_rebased_laptimes(laptimes,earliest)
ddx[-5:]

00:00:48
36:09.001
19:55.375
1:44.092
1:29.558
17:14.718
2:00.978
1:27.797
1:35.900
1:26.541
1:36.092
1:38.273
1:26.482
1:28.045
2:28.469
00:01:21
18:34.696
1:43.443
1:27.608
1:48.849
1:26.299
1:49.175
1:25.462
1:51.096
1:25.063
24:30.413
1:45.258
1:25.130
1:52.826
1:52.838
1:51.489
1:25.565
1:55.108
12:30.018
1:58.271
1:29.347
1:29.496
1:29.061
1:29.042
2:05.780
00:00:00
20:18.710
1:44.808
1:30.108
1:27.657
1:32.251
1:26.785
1:32.451
1:25.911
1:50.804
29:56.874
1:44.958
1:31.504
1:36.000
1:31.754
1:25.340
1:39.747
1:24.995
13:39.660
1:40.004
1:29.014
2:32.900
00:01:04
20:14.417
1:45.081
1:26.989
1:33.167
1:32.296
1:35.605
1:25.842
1:47.889
27:16.429
1:42.187
1:25.488
1:45.409
1:25.734
1:42.661
1:35.615
1:36.818
1:24.875
13:43.934
1:39.732
1:29.193
1:28.680
2:39.778
00:00:04
25:45.655
1:53.516
1:32.274
1:28.030
1:38.689
1:27.867
1:27.799
1:29.749
23:55.846
1:55.694
1:26.458
1:40.471
1:29.434
1:25.730
1:42.869
1:25.811
11:10.461
1:51.401
1:30.180
1:29.998
1:29.848
1:32.526
2:44.896
00:0

,lapNumber,laptime,name,number,pit,stime
471,25,1:29.549,Valtteri BOTTAS,77,False,89.549
472,26,1:29.674,Valtteri BOTTAS,77,False,89.674
473,27,1:29.548,Valtteri BOTTAS,77,False,89.548
474,28,1:29.345,Valtteri BOTTAS,77,False,89.345
475,29,2:43.078,Valtteri BOTTAS,77,False,163.078


In [56]:
laps_out_fn

'aus18_p1laptimes.csv'

In [57]:
ddx.to_csv(laps_out_fn,index=False)

In [58]:
ddx.head()

,lapNumber,laptime,name,number,pit,stime
0,1,00:00:48,Stoffel VANDOORNE,2,True,48.000
1,2,36:09.001,Stoffel VANDOORNE,2,True,2169.001
2,3,19:55.375,Stoffel VANDOORNE,2,True,1195.375
3,4,1:44.092,Stoffel VANDOORNE,2,False,104.092
4,5,1:29.558,Stoffel VANDOORNE,2,False,89.558


In [59]:
len(ddx['name'].unique())

20

In [60]:
laps_out_fn

'aus18_p1laptimes.csv'